In [ ]:
# =====================================================
# CNN SAJA - KEMIRIPAN
# RGB TETAP DIEKSTRAK
# HASIL DISIMPAN KE CSV
# =====================================================

import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# KONFIGURASI
# =========================
IMAGE_SIZE = (256, 256)

DATASET_DIR = "/content/drive/MyDrive/Kolong/dataset_citra"
CSV_FILE    = "/content/drive/MyDrive/Kolong/Dataset_skripsi.csv"
MODEL_FILE  = "/content/drive/MyDrive/Kolong/CNN_81%.h5"
TEST_DIR    = "/content/drive/MyDrive/Kolong/Citra tidak diketahui"

CNN_FEATURE_LAYER = "dense_16"

# =========================
# LOAD MODEL
# =========================
model = load_model(MODEL_FILE)

feature_extractor = Model(
    inputs=model.input[0],
    outputs=model.get_layer(CNN_FEATURE_LAYER).output
)

# =========================
# FUNGSI CNN
# =========================
def extract_cnn_feature(img_path):
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return feature_extractor.predict(img, verbose=0).flatten()

# =========================
# FUNGSI RGB PROPORSIONAL
# =========================
def extract_rgb_proportional(img_path):
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img = image.img_to_array(img) / 255.0
    R, G, B = np.mean(img[:,:,0]), np.mean(img[:,:,1]), np.mean(img[:,:,2])
    total = R + G + B
    return np.array([R/total, G/total, B/total]) if total != 0 else np.zeros(3)

# =========================
# DATABASE DATA DIKETAHUI
# =========================
df = pd.read_csv(CSV_FILE)

db_features, db_files, db_labels = [], [], []

for _, row in df.iterrows():
    img_path = os.path.join(DATASET_DIR, row["Tempat"])
    db_features.append(extract_cnn_feature(img_path))
    db_files.append(row["Tempat"])
    db_labels.append(row["Warna"])

db_features = np.array(db_features)

# =========================
# PENAMPUNG CSV  <<<<<< INI PENTING
# =========================
summary_results = []
detail_results  = []

# =========================
# PENGUJIAN
# =========================
top1_similarities = []
test_images = sorted(os.listdir(TEST_DIR))

for test_img in test_images:
    test_path = os.path.join(TEST_DIR, test_img)

    test_feat = extract_cnn_feature(test_path)
    _ = extract_rgb_proportional(test_path)

    similarities = cosine_similarity([test_feat], db_features)[0]
    best_idx = np.argmax(similarities)
    best_sim = similarities[best_idx]

    top1_similarities.append(best_sim)

    # ===== SIMPAN RINGKAS =====
    summary_results.append({
        "citra_uji": test_img,
        "paling_mirip": db_files[best_idx],
        "warna_prediksi": db_labels[best_idx],
        "top1_similarity": best_sim
    })

    # ===== SIMPAN DETAIL =====
    for i in range(len(db_files)):
        detail_results.append({
            "citra_uji": test_img,
            "citra_referensi": db_files[i],
            "warna_referensi": db_labels[i],
            "similarity": similarities[i]
        })

# =========================
# SIMPAN CSV  <<<<<< INI BAGIAN AKHIR
# =========================
pd.DataFrame(summary_results).to_csv(
    "/content/drive/MyDrive/Kolong/hasil_cnn_summary.csv",
    index=False
)

pd.DataFrame(detail_results).to_csv(
    "/content/drive/MyDrive/Kolong/hasil_cnn_detail.csv",
    index=False
)

# =========================
# AKURASI GLOBAL
# =========================
print(f"Akurasi Kemiripan CNN : {np.mean(top1_similarities)*100:.2f}%")
print("=== CNN SAJA SELESAI ===")

Akurasi Kemiripan CNN : 98.43%
=== CNN SAJA SELESAI ===
